### `shaprank` - An application to a _classification_ problem
In this notebook we run through a feature ranking example based on the `scikit-learn`'s dataset `breast cancer`. We rely on `catboost` to grow a simple tree-based model that consumes all raw input features and then use this model trained on default hyper-parameters to produce the (Tree)SHAP values consumed by `shaprank`.

For the sake of compactness, to drive the main points, we work only with the full dataset and take no splits.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging

logging.getLogger().setLevel(logging.INFO)

import shaprank
import shaprank.addons.explain

import examples

Load the `breast cancer` dataset - part of `scikit-learn`, - into the frame `df`. The variables `c_inputs` and `c_output` are, respectively, the list of input features' names and the name of the binary classification target.

In [ ]:
df, c_inputs, c_target = examples.load_dataset_breast_cancer()
c_output = "label"
df[c_output] = df[c_target] == "malignant"

In [ ]:
cb_model = examples.fit_catboost_classifier(df, c_inputs, c_output)

# concatenate the target column to the frame of SHAP values using `c_keep`
logging.info("Evaluating the SHAP values; find a few examples below.")
df_shap = shaprank.addons.explain.catboost.eval_shap_values(
    cb_model, df, c_keep=[c_output], prefix=""
)
df_shap.head(3)

### Greedy-search based feature ranking

Rank the input features using a "greedy search" algorithm that iteratively selects those features that provide the least contribution in terms of a given optimization objective. We inspect the results for `recall`.

In [ ]:
r_positives = df[c_output].mean()


result = shaprank.rank_classifier_features(
    df_shap,
    c_inputs,
    c_output,
    calib_metric="alert_rate",
    calib_metric_target=r_positives,
    eval_metric="recall",
    calib_metric_penalty=1,
    verbose=True,
)

According to the `shaprank` summary above, the model would be able nearly recall all examples at the target alert-rate used for calibration by consuming just 6 of the 30 features.

For the sake of comparison we then train models using the 6 top features selected by `shaprank` and the 6 top-raking features according to the "average absolute SHAP value" ranking produced by `shap`'s [Global Bar Plot](https://shap.readthedocs.io/en/latest/example_notebooks/api_examples/plots/bar.html#Global-bar-plot).

In [ ]:
(df_shap[c_inputs]).abs().mean(axis=0).sort_values(ascending=False)[:6]

In [ ]:
cb_model_featsel_shaprank = examples.fit_catboost_classifier(
    df,
    [
        "worst area",
        "mean concave points",
        "worst texture",
        "worst symmetry",
        "perimeter error",
        "worst fractal dimension",
    ],
    c_output,
)

In [ ]:
cb_model_featsel_shap = examples.fit_catboost_classifier(
    df,
    [
        "worst concave points",
        "mean concave points",
        "worst area",
        "worst radius",
        "worst perimeter",
        "worst concavity",
    ],
    c_output,
)

In this examples, the model trained on the reduced set of features ranked by `shaprank` achieves on average a loss `50%` smaller than with the alternative feature selection. 